## Getting started with local simulators

#### Braket

In [85]:
from braket import circuits as bk_circuits
from braket import devices as bk_devices

import numpy as np

In [86]:
bk_sim = bk_devices.LocalSimulator(backend="braket_dm") 

In [100]:
bk_circuit = bk_circuits.Circuit()

theta = [np.pi/4, np.pi/2, 3*np.pi/4, np.pi]

bk_circuit.h([0, 1])
bk_circuit.ry(0, theta[0])
bk_circuit.rz(1, theta[1])
bk_circuit.ry(0, theta[2])
bk_circuit.rz(1, theta[3])

print(bk_circuit)

T  : |0|    1    |   2    |
                           
q0 : -H-Ry(0.785)-Ry(2.36)-
                           
q1 : -H-Rz(1.57)--Rz(3.14)-

T  : |0|    1    |   2    |


In [101]:
bk_unitary = bk_circuits.unitary_calculation.calculate_unitary(bk_circuit.qubit_count, bk_circuit.instructions)

In [102]:
bk_sim_result = bk_dm.run(bk_circuit, shots=1024).result()

In [151]:
help(bk_sim_result)

Help on GateModelQuantumTaskResult in module braket.tasks.gate_model_quantum_task_result object:

class GateModelQuantumTaskResult(builtins.object)
 |  GateModelQuantumTaskResult(task_metadata: 'TaskMetadata', additional_metadata: 'AdditionalMetadata', result_types: 'List[ResultTypeValue]' = None, values: 'List[Any]' = None, measurements: 'np.ndarray' = None, measured_qubits: 'List[int]' = None, measurement_counts: 'Counter' = None, measurement_probabilities: 'Dict[str, float]' = None, measurements_copied_from_device: 'bool' = None, measurement_counts_copied_from_device: 'bool' = None, measurement_probabilities_copied_from_device: 'bool' = None, _result_types_indices: 'Dict[str, int]' = None) -> None
 |  
 |  Result of a gate model quantum task execution. This class is intended
 |  to be initialized by a QuantumTask class.
 |  
 |  Args:
 |      task_metadata (TaskMetadata): Task metadata.
 |      additional_metadata (AdditionalMetadata): Additional metadata about the task
 |      resu

In [163]:
bk_sim_result.measurements
# bk_sim_result.measurement_counts
# bk_sim_result.measurement_probabilities

array([[0, 0],
       [1, 0],
       [0, 0],
       ...,
       [1, 1],
       [1, 1],
       [0, 1]])

#### Cirq

In [98]:
import cirq

In [111]:
cirq_sim = cirq.DensityMatrixSimulator()

In [142]:
qs = cirq.LineQubit.range(2)
# qs.reverse()

cirq_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    cirq.ry(rads=theta[0])(qs[0]),
    cirq.rz(rads=theta[1])(qs[1]),
    cirq.ry(rads=theta[2])(qs[0]),
    cirq.rz(rads=theta[3])(qs[1]),
    cirq.measure(q0, key=str(qs[0].x)),
    cirq.measure(q1, key=str(qs[1].x)),
)

print(cirq_circuit)

0: ───H───Ry(0.25π)───Ry(0.75π)───M───

1: ───H───Rz(0.5π)────Rz(π)───────M───


In [127]:
cirq_unitary = cirq_circuit.unitary()

In [156]:
cirq_sim_result = cirq_sim.run(cirq_circuit, repetitions=1024)

In [164]:
cirq_sim_result.measurements
# cirq_sim_result.data
# cirq_sim_result.histogram
# cirq_sim_result.multi_measurement_histogram(keys=cirq_sim_result.measurements.keys())

{'0': array([[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1]], dtype=int8),
 '1': array([[1],
        [1],
        [0],
        ...,
        [1],
        [1],
        [0]], dtype=int8)}

Counter({(1, 1): 261, (0, 1): 257, (0, 0): 263, (1, 0): 243})

In [132]:
help(cirq_sim_result)

Help on Result in module cirq.study.result object:

class Result(builtins.object)
 |  Result(*, params: cirq.study.resolver.ParamResolver, measurements: Dict[str, numpy.ndarray]) -> None
 |  
 |  The results of multiple executions of a circuit with fixed parameters.
 |  Stored as a Pandas DataFrame that can be accessed through the "data"
 |  attribute. The repetition number is the row index and measurement keys
 |  are the columns of the DataFrame. Each element is a big endian integer
 |  representation of measurement outcomes for the measurement key in that
 |  repetition.  See `cirq.big_endian_int_to_bits` and similar functions
 |  for how to convert this integer into bits.
 |  
 |  Attributes:
 |      params: A ParamResolver of settings used when sampling result.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other: 'cirq.Result') -> 'cirq.Result'
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, *, params: cirq.study.resolver.ParamResolver, mea

In [129]:
cirq_sim_meas = cirq_sim_result.measurements

In [130]:
print(cirq_sim_meas)

{'0': array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int8), '1': array([[1],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int8)}


## What are the available quantum backends?

#### Braket QPU

Note: To interact with the Braket API in Python, you must first `pip install boto3`, and create local credential/config files using authentication from your AWS account, see [Boto3 Docs](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration).

In [1]:
import boto3

In [2]:
client = boto3.client('braket')

In [3]:
client.search_devices(filters=[])

{'ResponseMetadata': {'RequestId': '122c1474-fea0-4ea4-a5ef-db2fc3a4fb35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 28 Jan 2022 19:55:49 GMT',
   'content-type': 'application/json',
   'content-length': '25091',
   'connection': 'keep-alive',
   'x-amzn-requestid': '122c1474-fea0-4ea4-a5ef-db2fc3a4fb35',
   'access-control-allow-origin': '*',
   'strict-transport-security': 'max-age=63072000; includeSubDomains; preload',
   'access-control-allow-headers': '*,authorization,date,x-amz-date,x-amz-security-token,x-amz-target,content-type,x-amz-content-sha256,x-amz-user-agent,x-amzn-platform-id,x-amzn-trace-id',
   'x-amz-apigw-id': 'MrCK8ErMIAMFucg=',
   'access-control-allow-methods': 'OPTIONS,GET,PUT,POST,DELETE',
   'access-control-expose-headers': 'x-amzn-errortype,x-amzn-requestid,x-amzn-errormessage,x-amzn-trace-id,x-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-61f44a45-2eb4c0e93447054271e705a4'},
  'RetryAttempts': 0},
 'devices': [{'deviceArn': 'arn:aws:braket:

In [11]:
from braket.circuits import Circuit as BkCircuit
from braket.devices import LocalSimulator as BkSimulator
from braket.aws import AwsDevice

import numpy as np

In [20]:
bk_circuit = BraketCircuit()

bk_circuit.h([0, 1, 2, 3])
bk_circuit.rx(0, np.pi / 4)
bk_circuit.ry(1, np.pi / 2)
bk_circuit.rz(2, 3 * np.pi / 4)
bk_circuit.phaseshift(3, np.pi / 8)
bk_circuit.cnot(0, 2)
bk_circuit.cnot(1, 3)
bk_circuit.swap(1, 2)

print(bk_circuit)

T  : |0|     1      | 2 | 3  |
                              
q0 : -H-Rx(0.785)----C--------
                     |        
q1 : -H-Ry(1.57)-----|-C-SWAP-
                     | | |    
q2 : -H-Rz(2.36)-----X-|-SWAP-
                       |      
q3 : -H-PHASE(0.393)---X------

T  : |0|     1      | 2 | 3  |


In [44]:
aws_dm1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

In [45]:
# Please enter the S3 bucket you created during onboarding in the code below

my_bucket = os.environ["s3_bucket"]
my_prefix = os.environ["s3_prefix"]

# my_bucket = "amazon-braket-Your-Bucket-Name" # the name of the bucket
# my_prefix = "Your-Folder-Name" # the name of the folder in the bucket

s3_folder = (my_bucket, my_prefix)

In [46]:
aws_dm1_job = aws_dm_sim.run(bk_circuit, s3_folder, shots=1024)

In [47]:
aws_dm1_job.metadata()["status" ]

'CREATED'

In [48]:
aws_dm1_result = job.result()

In [49]:
aws_dm1_counts = aws_dm1_result.measurement_counts

In [58]:
print(aws_dm1_counts) # <class 'collections.Counter'>

Counter({'1011': 138, '0011': 134, '1110': 128, '0110': 127, '1111': 127, '0111': 124, '1010': 124, '0010': 122})


In [24]:
import qbraid

In [59]:
qbraid.get_devices()

Provider,Name,qBraid ID,Status
AWS,Braket Default Simulator,aws_braket_default_sim,●
AWS,Density Matrix Simulator,aws_dm_sim,●
AWS,State Vector Simulator,aws_sv_sim,●
AWS,Tensor Network Simulator,aws_tn_sim,●
D-Wave,Advantage_system4,aws_dwave_advantage_system4,●
D-Wave,DW_2000Q_6,aws_dwave_2000Q_6,●
Google,Bristlecone,google_bristlecone,○
Google,Cirq Density Matrix Simulator,google_cirq_dm_sim,●
Google,Cirq Sparse Simulator,google_cirq_sparse_sim,●
Google,Foxtail,google_foxtail,○


In [26]:
aws_device = device_wrapper("aws_dm_sim")

In [29]:
aws_device._s3_location

('amazon-braket-592242689881', 'simulator-output')

In [30]:
import os
 = 'amazon-braket-592242689881'
 = 'simulator-output'

In [ ]:
from qbraid import (
    get_devices, 
    refresh_devices, 
    random_circuit, 
    device_wrapper, 
)

Get a complete list of devices available through qBraid using the get_devices function

In [ ]:
get_devices()

All device status's are updated every hour. To manually refresh device status's, use refresh_devices

In [ ]:
refresh_devices()

Filter your device search with many different advanced query options! 

For example, let's find all gate-based QPUs with at least 5 qubits that are online:

In [ ]:
get_devices({"paradigm": "gate-based", "type": "QPU", "qubits": {"$gte": 5}, "status": "ONLINE"})

This time, let's search for state vector simulators available through AWS or IBM

In [ ]:
get_devices({"type": "Simulator", "name": {"$regex": "Density Matrix"}, "vendor": {"$in": ["AWS", "Google"]}})

The qBraid SDK makes running quantum circuits on any device extremely easy. 

Simply copy the desired qBraid ID into the device wrapper function and your on your way

In [ ]:
aws_device = device_wrapper("aws_dm_sim")

In [ ]:
google_device = device_wrapper("google_cirq_dm_sim")

In [ ]:
ibm_device = device_wrapper("ibm_q_lima") 

In [ ]:
ibm_device.info

In [ ]:
circuit = random_circuit("qiskit", num_qubits=3, depth=5, measure=True)
print(circuit)

In [ ]:
shots=1000

In [ ]:
aws_job = aws_device.run(circuit, shots=shots)

In [ ]:
google_job = google_device.run(circuit, shots=shots)

In [ ]:
ibm_job = ibm_device.run(circuit, shots=shots)

In [ ]:
aws_job.id

In [ ]:
# ibm_job.device.pending_jobs()
ibm_job.status()

In [ ]:
ibm_result = ibm_job.result()

In [ ]:
google_result = google_job.result()

In [ ]:
aws_result = aws_job.result()

In [ ]:
aws_result.measurement_counts()

In [ ]:
google_result.measurement_counts()

In [ ]:
ibm_result.measurement_counts()

In [ ]:
ibm_result.plot_counts()

In [ ]:
aws_result.plot_counts()

In [ ]:
google_result.plot_counts()